In [1]:
import parser_funcs
import numpy as np
import os
import math
from scipy.special import kn
from scipy.interpolate import interp1d
from scipy.interpolate import SmoothBivariateSpline
from scipy.optimize import brentq
from scipy.integrate import quad
from scipy.interpolate import griddata
from scipy.optimize import minimize_scalar
from scipy.optimize import root_scalar
import sys
import glob
import random
#import matplotlib.pyplot as plt

In [2]:
#Units
gev=1;mev=1e-3*gev;kev=1e-6*gev;

#constants
hbar=float(1.054*1e-34/(1.6e-19)/(1e9));
speed_of_light=3e8;conversion=hbar**2*speed_of_light**2;
alpha_em=1.0/137.035999074;

#masses
mp=0.938272046*gev;melec=511*kev;mmuon=105.658*mev;
mpi=134.9767*mev;mpip=139.57018*mev;mkaon=0.493667*gev;
mj_psi=3.097*gev;

In [3]:
def mom(arr):
    return math.sqrt(arr[3]**2+arr[1]**2+arr[2]**2)
    
def theta(arr):
    return math.acos(arr[3]/mom(arr))

def phi(arr):
    return math.atan(arr[0]/arr[1])

def cos_theta(arr):
    return arr[3]/mom(arr)

def theta_dif(arr1,arr2):
    return math.acos(dp(arr1[1:4],arr2[1:4])/mom(arr1)/mom(arr2))

#Expect each element of arr to be 4 elements. Expects px py pz E.
def invariant_mass(arr):
    tot = [0,0,0,0]
    for line in arr:
        for i in range(4):
            tot[i]+=line[i]
    return tot[0]**2-tot[3]**2-tot[1]**2-tot[2]**2

def cos_angle_between(arr1,arr2):
    return (arr1[3]*arr2[3]+arr1[1]*arr2[1]+arr1[2]*arr2[2])/mom(arr1)/mom(arr2)

In [4]:
def momvec(arr):
    return [arr[1],arr[2],arr[3]]
def startvec(arr):
    return [arr[5],arr[6],arr[7]]
def decaypoint(arr):
    return math.sqrt(arr[9]**2+arr[10]**2+arr[11]**2)
def dp(a1,a2):
    tot=0
    for i in range(len(a1)):
        tot+=a1[i]*a2[i]
    return tot
def veclen(a1):
    return dp(a1,a1)**0.5
def sub(a1,a2):
    return [a1[i]-a2[i] for i in range(len(a1))]
def add(a1,a2):
    return [a1[i]+a2[i] for i in range(len(a1))]
def mul(c,a1):
    return [c*a1[i] for i in range(len(a1))]
def plane_cross(p,o,r,n):
    bot=dp(n,p)
    if bot == 0:
        return -1
    a = dp(n,sub(r,o))/bot
    point = add(o,mul(a,p))
    R=sub(r,point)
    return point,dp(R,R)**0.5

In [5]:
def record_list(outfile,data):
    with open(outfile,'w') as w:
        for line in data:
            tmpstr=""
            for elem in line:
                tmpstr+=str(elem)+' '
            tmpstr=tmpstr[:-1]
            tmpstr+='\n'
            w.write(tmpstr)
def append_list(outfile,data):
    with open(outfile,'a') as w:
        for line in data:
            tmpstr=""
            for elem in line:
                tmpstr+=str(elem)+' '
            tmpstr=tmpstr[:-1]
            tmpstr+='\n'
            w.write(tmpstr)

In [27]:
def format_rratio(rratio):
    for line in rratio:
        linearr=line.split()
        if len(linearr)>=14:
            try:
                yield [float(linearr[0]),float(linearr[3])]
            except ValueError:
                continue

with open('data/rratio_raw.dat','r') as infile:
    rratio1=infile.read()
    
rratio_rough = rratio1.splitlines()
rratio_clean=np.array(list(format_rratio(rratio_rough)),dtype=float)
f_rratio = interp1d(rratio_clean[:,0],rratio_clean[:,1])
def rratio(s):
    if s<rratio_clean[0,0]:
        return 0
    else:
        return f_rratio(s)

In [26]:
def GammaV(kappa, mv, alpha_p=0, mx=100):
    term = 0;
    if mv>2*mx:
        term += alpha_p*(mv*mv-4*mx*mx)*math.sqrt(mv*mv/4.0-mx*mx)
    if mv>2*melec:
        term += 4*pow(kappa,2)*alpha_em*(2*pow(melec,2)+mv*mv)*math.sqrt(mv*mv/4.0-pow(melec,2))
    if mv>2*mmuon:
        term += 4*pow(kappa,2)*alpha_em*(2*pow(mmuon,2)+mv*mv)*math.sqrt(mv*mv/4.0-pow(mmuon,2))*(1+rratio(mv))
    return 1.0/(6.0*mv*mv)*(term)

In [8]:
def Gamma_V_to_ll(kappa, mv, ml, alpha_p=0, mx=100):
    if mv>2*ml:
        return 4*pow(kappa,2)*alpha_em*(2*pow(ml,2)+mv*mv)*math.sqrt(mv*mv/4.0-pow(ml,2))/(6.0*mv*mv)
    else:
        return 0
def Br_V_to_leptons(kappa,mv,alpha_p=0,mx=100):
    GV=GammaV(kappa, mv, alpha_p=alpha_p, mx=mx)
    if GV<=0:
        return 0
    return (Gamma_V_to_ll(kappa,mv,melec,alpha_p=alpha_p,mx=mx)+Gamma_V_to_ll(kappa,mv,mmuon,alpha_p=alpha_p,mx=mx))/GammaV(kappa, mv, alpha_p=alpha_p, mx=mx)

In [9]:
#l_i = dist_i/speed_of_light/lifetime*mass
def dec_prob(l1,l2,momentum):
    return math.exp(-l1/momentum)*(1-math.exp(-l2/momentum))
def calc_mean_events(dat,dist,eps,epset,mv,events):
    if GammaV(eps,mv) <= 0:
        return 0.0
    lifetime = hbar/GammaV(eps,mv)
    l1 = dist/speed_of_light/lifetime*mv
    l2 = 1.0/speed_of_light/lifetime*mv
    tot=0
    for u in dat:
        tot+=dec_prob(l1,u[2]*l2,u[0])
    return tot/len(dat)*events*(eps/epset)**2

In [10]:
def process_infile_data(file,cut):
    with open(file) as infile:
        dat = infile.read()
        dat = dat.splitlines()
        sumline = dat[-1].split()
        dat = dat[:-1]
        dat = [u.split() for u in dat]
        tot=len(dat)
        dat = cut(dat)
        dat = [[float(u[4]),float(u[5]),float(u[6]),float(u[3]),float(u[1])] for u in dat]
        dat = [[math.sqrt(u[0]**2+u[1]**2+u[2]**2),u[3],u[4]] for u in dat]
        mv = float(sumline[1])
        eps = float(sumline[4])
        intersects=len(dat)/tot*float(sumline[-1])*float(sumline[-3])
        return(mv,eps,intersects,dat)

In [11]:
nucal_dist=64;nucal_eff=0.7;
#nucal_evnt=4.4894
nucal_evnt=7.28
def nucal_cut(dat):
    return [u for u in dat if float(u[3])>3]
def nucal_cut_2(dat):
    return [u for u in dat if float(u[3])>10]

In [12]:
def calc_params_nucal(file):
    mv,eps_eval,events,dat=process_infile_data(file,nucal_cut)
    #if mv>0.8:
    #    return [mv,-1,-1]
    evnt_rate = lambda eps : -1.0*calc_mean_events(dat,nucal_dist,10**eps,eps_eval,mv,events*nucal_eff*Br_V_to_leptons(eps,mv))
    diff_from_evnt = lambda eps : abs(evnt_rate(eps)+nucal_evnt)
    opt=minimize_scalar(evnt_rate,bounds=(-8,-3),method='bounded',options={'maxiter':500})
    if opt.fun>-nucal_evnt:
        return [mv,-1,-1]
    x=10
    opt2=minimize_scalar(diff_from_evnt,bounds=(-8,opt.x),method='bounded',options={'maxiter':500})
    opt3=minimize_scalar(diff_from_evnt,bounds=(opt.x,opt.x+1),method='bounded',options={'maxiter':500})
    if opt3.fun > 0.1:
        return [mv,10**opt2.x,-1]
    return [mv,10**opt2.x,10**opt3.x]

In [52]:
def Event_Parser(event_file, cut):
    with open(event_file) as ef:
        dat=ef.read().splitlines()
        run_num=''
        for line in dat:
            line=line.split()
            if len(line)==2 and line[0]=="Run":
                run_num=line[1]                
                break
    dat2 = [line.split() for line in dat]
    sum_line= dat2[-1]
    weight_tab=[float(line[2]) for line in dat2 if len(line)==3 and line[0]=="event"]
    dp_tab=[[float(line[i]) for i in range(1,len(line))] for line in dat2 if len(line)>1\
                and line[0]=='Dark_Photon']
    lep = [[float(line[i]) for i in range(1,len(line))]+[line[0]] for line in dat2 if len(line)>1\
                and (line[0]=='Electron' or line[0]=='Muon' or line[0]=='Hadronic_Stuff')]
    antilep = [[float(line[i]) for i in range(1,len(line))]+[line[0]] for line in dat2 if len(line)>1 and (line[0]=='Positron' or line[0]=='Anti-muon' or line[0]=='Hadronic_Stuff')]
    mv = float(sum_line[2])
    nevents = float(sum_line[1])
    POT = float(sum_line[7])
    alpha_D=float(sum_line[5])
    eps=float(sum_line[4])
    dp_lifetime=hbar/GammaV(eps,mv)
    eff=float(sum_line[8])
    nevents,weight_tab,dp_tab,lep_tab,antilep_tab=cut(nevents,weight_tab,dp_tab,lep,antilep)
    return [float(mv),alpha_D,eps,eff,dp_lifetime,nevents,weight_tab,dp_tab,lep,antilep]

In [14]:
#l_i = dist_i/speed_of_light/lifetime*mass
def dec_prob(l1,l2):
    return math.exp(-l1)-math.exp(-l2)
def dec_loc(l1,l2):
    try:
        return -math.log(math.exp(-l1)-random.random()*(math.exp(-l1)-math.exp(-l2)))
    except:
        return l1
def solve_dec_loc(l1,l2,pos):
    return (10**pos-math.exp(-l1))/(math.exp(-l2)-math.exp(-l1))
def dec_loc_set(l1,l2,pos):
    try:
        return -math.log(math.exp(-l1)-pos*(math.exp(-l1)-math.exp(-l2)))
    except:
        return l1
def Return_True(dat,dm,lep,alep,mx2,l):
    return True
def Rescale_Events(dat,dp,lep,alep,mv,lifetime,tot_weight,eps,epset,events,cut,state=None):
    if state is not None:
        random.setstate(state)
    prob=0
    life=lifetime*(epset/eps)**2
    l=1.0/speed_of_light/life*mv
    for i,u in enumerate(dat):
        ptmp=dec_prob(u[1]*l/u[0],u[2]*l/u[0])
        if cut(u,dp[i],lep[i],alep[i],mv,l):
            prob+=ptmp
        else:
            continue
    return prob/tot_weight*events*(eps/epset)**2

In [59]:
def calc_params(file,evnt,cut,cut2=Return_True):
    try:
        mv,alpha_D,eps_eval,eff,lifetime,nevents,weight_tab,dp_dat,lep,alep=Event_Parser(file,cut)
    except:
        print("Exception", file)
        return [-1,-1,-1]
    if nevents==0:
        return [-1,-1,-1]
    dat = [[mom(u),u[14],u[15]] for u in dp_dat]
    tot_weight=0
    for i in weight_tab:
        tot_weight+=i
    state=random.getstate()
    evnt_rate = lambda eps : -1.0*Rescale_Events(dat,dp_dat,lep,alep,mv,lifetime,tot_weight,10**eps,eps_eval,nevents,cut2,state=state)
    diff_from_evnt = lambda eps : abs(evnt_rate(eps)+evnt)
    diff_from_evnt_2 = lambda eps : evnt-Rescale_Events(dat,dp_dat,lep,alep,mv,lifetime,tot_weight,eps,eps_eval,nevents,cut2,state=state)
    opt=minimize_scalar(evnt_rate,bounds=(-10,0),method='bounded')
    if opt.fun>-evnt:
        return [mv,-1,-1]
    
    opt2=minimize_scalar(diff_from_evnt,bounds=(-10,opt.x),method='bounded')

    x=opt.x
    xstep=0.1
    while diff_from_evnt_2(10**x)<0:
        x+=xstep
    
    sol = root_scalar(diff_from_evnt_2, bracket=[10**(x-xstep), 10**x], method='brentq')    
    
    return [mv,10**opt2.x,sol.root]

In [50]:
NA62_energy_cut=3
NA62_efficiency=1
NA62_evnt=2.3
NA62_KLR=[0,0,217]
NA62_n=[0,0,1]
NA62_pot_suppression=1.3e16/1e18
leph=[]
aleph=[]
wh=[]
def NA62_Cut(nevents,weight_tab,dp_tab,lep_tab,antilep_tab):
    pass_index=[]
    #print(len(lep_tab),len(weight_tab))
    for i in range(len(weight_tab)):
        if lep_tab[i][-1]=="Hadronic_Stuff":
            continue
        energy = lep_tab[i][0]+antilep_tab[i][0]
        if energy>NA62_energy_cut:
            pointl,Rl=plane_cross(momvec(lep_tab[i]),startvec(lep_tab[i]),NA62_KLR,NA62_n)
            #print(pointl,Rl)
            pointa,Ra=plane_cross(momvec(antilep_tab[i]),startvec(antilep_tab[i]),NA62_KLR,NA62_n)
            #print(pointa,Ra,veclen(sub(pointl,pointa)))
            if Ra>0.15 and Rl>0.15 and Ra<1 and Rl<1 and veclen(sub(pointl,pointa))>0.1:
                pass_index.append(i)
    nevents*=len(pass_index)/len(weight_tab)*NA62_efficiency
    weight_tab = [weight_tab[i] for i in pass_index]
    dp_tab = [dp_tab[i] for i in pass_index]
    lep_tab = [lep_tab[i] for i in pass_index]
    antilep_tab = [antilep_tab[i] for i in pass_index]
    return nevents,weight_tab,dp_tab,lep_tab,antilep_tab

In [17]:
#Taking advantage of the fact that I can only have one dm2 decay per event.
CHARM_energy_cut=3
charm_evnt=2.3;
CHARM_efficiency=0.6
charm_n=[0,0,1]
charm_det=[5,0,497.5]
def CHARM_Cut(nevents,weight_tab,dm2_tab,lep_tab,antilep_tab):
    pass_index=[]
    for i in range(len(weight_tab)):
        if lep_tab[i][-1]=="Hadronic_Stuff":
            continue
        energy=lep_tab[i][0]+antilep_tab[i][0]
        if energy>CHARM_energy_cut:
            pointl,Rl=plane_cross(momvec(lep_tab[i]),startvec(lep_tab[i]),charm_det,charm_n)
            if abs(pointl[0]-5)>3 or abs(pointl[1])>3:
                continue
            pointa,Ra=plane_cross(momvec(antilep_tab[i]),startvec(antilep_tab[i]),charm_det,charm_n)
            if abs(pointa[0]-5)>3 or abs(pointa[1])>3:
                continue
            pass_index.append(i)
    nevents*=len(pass_index)/len(weight_tab)*CHARM_efficiency
    weight_tab = [weight_tab[i] for i in pass_index]
    dm2_tab = [dm2_tab[i] for i in pass_index]
    lep_tab = [lep_tab[i] for i in pass_index]
    antilep_tab = [antilep_tab[i] for i in pass_index]
    return nevents,weight_tab,dm2_tab,lep_tab,antilep_tab

In [18]:
Nucal_efficiency=0.7
Nucal_energy_cut_1=3
Nucal_evnt_1=6.06
Nucal_angle=0.05
def Nucal_Cut_1(nevents,weight_tab,dm2_tab,lep_tab,antilep_tab):
    pass_index=[]
    for i in range(len(weight_tab)):
        energy=lep_tab[i][0]+antilep_tab[i][0]
        if lep_tab[i][-1]=="Electron" and energy>Nucal_energy_cut_1 and theta(lep_tab[i]) < Nucal_angle and theta(antilep_tab[i]) < Nucal_angle:
            pass_index.append(i)
    nevents*=len(pass_index)/len(weight_tab)*Nucal_efficiency
    weight_tab = [weight_tab[i] for i in pass_index]
    dm2_tab = [dm2_tab[i] for i in pass_index]
    lep_tab = [lep_tab[i] for i in pass_index]
    antilep_tab = [antilep_tab[i] for i in pass_index]
    return nevents,weight_tab,dm2_tab,lep_tab,antilep_tab

Nucal_energy_cut_2=10
Nucal_evnt_2=3.64
def Nucal_Cut_2(nevents,weight_tab,dm2_tab,lep_tab,antilep_tab):
    pass_index=[]
    for i in range(len(weight_tab)):
        energy=lep_tab[i][0]+antilep_tab[i][0]
        if lep_tab[i][-1]=="Electron" and energy>Nucal_energy_cut_2 and theta(lep_tab[i]) < Nucal_angle and theta(antilep_tab[i]) < Nucal_angle:
            pass_index.append(i)
    nevents*=len(pass_index)/len(weight_tab)*Nucal_efficiency
    weight_tab = [weight_tab[i] for i in pass_index]
    dm2_tab = [dm2_tab[i] for i in pass_index]
    lep_tab = [lep_tab[i] for i in pass_index]
    antilep_tab = [antilep_tab[i] for i in pass_index]
    return nevents,weight_tab,dm2_tab,lep_tab,antilep_tab

In [47]:
DUNE_efficiency=1
DUNE_evnt=10
elec_kinetic_cut=0.14*mev
muon_kinetic_cut=1.8*mev
def DUNE_cut(nevents,weight_tab,dm2_tab,lep_tab,antilep_tab):
    pass_index=[]
    for i in range(len(weight_tab)):
        if lep_tab[i][-1]=="Electron" and lep_tab[i][13]>elec_kinetic_cut and antilep_tab[i][13]>elec_kinetic_cut:
            pass_index.append(i)
        elif lep_tab[i][-1]=="Muon" and lep_tab[i][13]>muon_kinetic_cut and antilep_tab[i][13]>muon_kinetic_cut:
            pass_index.append(i)
    nevents*=len(pass_index)/len(weight_tab)*DUNE_efficiency
    weight_tab = [weight_tab[i] for i in pass_index]
    dm2_tab = [dm2_tab[i] for i in pass_index]
    lep_tab = [lep_tab[i] for i in pass_index]
    antilep_tab = [antilep_tab[i] for i in pass_index]
    return nevents,weight_tab,dm2_tab,lep_tab,antilep_tab

In [28]:
#Taking advantage of the fact that I can only have one dm2 decay per event.
Seaquest_energy_cut=0#Uuuuh, maybe not?
Seaquest_efficiency=1
Seaquest_evnt=10
Seaquest_evnt_high=52
Seaquest_evnt_med=16
seaquest_low_pot_factor=1e18/1e20
def Seaquest_Cut(nevents,weight_tab,dm2_tab,lep_tab,antilep_tab):
    pass_index=[]
    for i in range(len(weight_tab)):
        if lep_tab[i][3]<0 or antilep_tab[i][3]<0:
            continue
        #energy=lep_tab[i][0]+antilep_tab[i][0]
        #if energy>Seaquest_energy_cut:
        if lep_tab[i][0]>Seaquest_energy_cut and antilep_tab[i][0]>Seaquest_energy_cut:
            pass_index.append(i)
    nevents*=len(pass_index)/len(weight_tab)*Seaquest_efficiency
    weight_tab = [weight_tab[i] for i in pass_index]
    dm2_tab = [dm2_tab[i] for i in pass_index]
    lep_tab = [lep_tab[i] for i in pass_index]
    antilep_tab = [antilep_tab[i] for i in pass_index]
    return nevents,weight_tab,dm2_tab,lep_tab,antilep_tab

In [22]:
seaquest_plane_n=[0,0,1]
seaquest_KMAG=[0,0,12]
seaquest_det=[0,0,18.5]
def Seaquest_Cut_2(dat,dm,lep,alep,mx2,l):
    loc_scaling=dec_loc(dat[1]*l/dat[0],dat[2]*l/dat[0])*l/mom(dm)
    decay_point = [dm[1+i]*loc_scaling+dm[5+i] for i in range(3)]
    lmomvec=momvec(lep);amomvec=momvec(alep);
    kick=0
    if decay_point[2]<seaquest_KMAG[2]:
        kick=0.4*min(3,12-decay_point[2])/3.0
        point1l,R=plane_cross(lmomvec,decay_point,seaquest_KMAG,seaquest_plane_n)
        lmomvec[0]+=kick
        point2l,R=plane_cross(lmomvec,point1l,seaquest_det,seaquest_plane_n)
    else:
        point2l,R=plane_cross(lmomvec,decay_point,seaquest_det,seaquest_plane_n)
    if abs(point2l[0])-1>0 or abs(point2l[1])-1>0:
        return False
    if decay_point[2]<seaquest_KMAG[2]:
        point1a,R=plane_cross(amomvec,decay_point,seaquest_KMAG,seaquest_plane_n)
        amomvec[0]+=kick
        point2a,R=plane_cross(amomvec,point1a,seaquest_det,seaquest_plane_n)
    else:
        point2a,R=plane_cross(amomvec,decay_point,seaquest_det,seaquest_plane_n)
    if abs(point2a[0])-1>0 or abs(point2a[1])-1>0:
        return False
    return True

In [27]:
Seaquest_efficiency=1
Seaquest_evnt=10
seaquest_low_pot_factor=1e18/1e20
Seaquest_no_kmag_energy_cut=3
seaquest_plane_n=[0,0,1]
seaquest_det=[0,0,18.5]
def Seaquest_Cut_no_kmag(nevents,weight_tab,dp_tab,lep_tab,antilep_tab):
    pass_index=[]
    for i in range(len(weight_tab)):
        if lep_tab[i][3]<0 or antilep_tab[i][3]<0:
            continue
        energy=lep_tab[i][0]+antilep_tab[i][0]
        if energy>Seaquest_no_kmag_energy_cut:
        #if lep_tab[i][0]>Seaquest_energy_cut and antilep_tab[i][0]>Seaquest_energy_cut:
            pass_index.append(i)
    nevents*=len(pass_index)/len(weight_tab)*Seaquest_efficiency
    weight_tab = [weight_tab[i] for i in pass_index]
    dp_tab = [dp_tab[i] for i in pass_index]
    lep_tab = [lep_tab[i] for i in pass_index]
    antilep_tab = [antilep_tab[i] for i in pass_index]
    return nevents,weight_tab,dp_tab,lep_tab,antilep_tab
def Seaquest_Cut_2_no_kmag(dat,dp,lep,alep,mv,l):
    loc_scaling=dec_loc(dat[1]*l/dat[0],dat[2]*l/dat[0])*l/mom(dp)
    decay_point = [dp[1+i]*loc_scaling+dp[5+i] for i in range(3)]
    lmomvec=momvec(lep);amomvec=momvec(alep);
    point1l,R=plane_cross(lmomvec,decay_point,seaquest_det,seaquest_plane_n)
    if abs(point1l[0])-1>0 or abs(point1l[1])-1>0:
        return False
    point1a,R=plane_cross(amomvec,decay_point,seaquest_det,seaquest_plane_n)
    if abs(point1a[0])-1>0 or abs(point1a[1])-1>0:
        return False
    return True

In [62]:
files=glob.glob('Visible_Dark_Photon/dune/dune_*')
bounds=[calc_params(file,DUNE_evnt,DUNE_cut) for file in files]
bounds = [x for x in bounds if x[0]!=-1 and (x[1]!=-1 and x[2]!=-1)]
bounds.sort()
record_list("Dune_dark_photon.dat",bounds)

Exception Visible_Dark_Photon/dune/dune_1.dat


In [58]:
calc_params('Visible_Dark_Photon/dune/dune_620.dat',DUNE_evnt,DUNE_cut)

[0.62, 2.048541275081858e-08, 2.7266586634793726e-07]

In [43]:
files=glob.glob('Visible_Dark_Photon/dune/dune_*')
bounds=[calc_params(file,Seaquest_evnt_high,Seaquest_Cut,Seaquest_Cut_2) for file in files]
bounds = [x for x in bounds if x[0]!=-1 and (x[1]!=-1 and x[2]!=-1)]
bounds.sort()
record_list("Dark_Photon_Seaquest_with_kmag.dat",bounds)

NameError: name 'Seaquest_evnt_high' is not defined

In [31]:
files=glob.glob('Visible_Dark_Photon/seaquest/seaquest_*')
bounds=[calc_params(file,Seaquest_evnt_med,Seaquest_Cut,Seaquest_Cut_2) for file in files]
bounds = [x for x in bounds if x[0]!=-1 and (x[1]!=-1 and x[2]!=-1)]
bounds.sort()
record_list("Dark_Photon_Seaquest_no_kmag.dat",bounds)

In [37]:
files=glob.glob('Visible_Dark_Photon/seaquest/seaquest_*')
bounds=[calc_params(file,Seaquest_evnt/seaquest_low_pot_factor,Seaquest_Cut,Seaquest_Cut_2) for file in files]
bounds = [x for x in bounds if x[0]!=-1 and (x[1]!=-1 and x[2]!=-1)]
bounds.sort()
record_list("Dark_Photon_Seaquest_with_kmag_low_POT.dat",bounds)

In [26]:
mvlist=['1800','1850','1900','1950']
files2=list()
files=glob.glob('Visible_Dark_Photon/seaquest_extended/seaquest_*')
for i in files:
    if i.split('_')[4] in mvlist:
        files2+=[i]
bounds=[calc_params(file,Seaquest_evnt,Seaquest_Cut_no_kmag,Seaquest_Cut_2_no_kmag) for file in files2]
bounds = [x for x in bounds if x[0]!=-1 and (x[1]!=-1 and x[2]!=-1)]
bounds.sort()

In [30]:
files=glob.glob('Visible_Dark_Photon/seaquest_extended/seaquest_*')
bounds=[calc_params(file,Seaquest_evnt_med,Seaquest_Cut_no_kmag,Seaquest_Cut_2_no_kmag) for file in files]
bounds = [x for x in bounds if x[0]!=-1 and (x[1]!=-1 and x[2]!=-1)]
bounds.sort()
record_list("Dark_Photon_Seaquest_Extended.dat",bounds)

In [27]:
bounds

[[1.8, 1.3655887959518137e-07, 3.4057233128413045e-07],
 [1.85, 1.6876667321847775e-07, 3.277512949927897e-07],
 [1.9, 2.137143530528175e-07, 2.7098611941494897e-07]]

In [29]:
append_list('Dark_Photon_Seaquest_Extended_0GeV.dat',bounds)

In [23]:
files=glob.glob('Visible_Dark_Photon/seaquest_extended/seaquest_*')
bounds=[calc_params(file,Seaquest_evnt/seaquest_low_pot_factor,Seaquest_Cut_no_kmag,Seaquest_Cut_2_no_kmag) for file in files]
bounds = [x for x in bounds if x[0]!=-1 and (x[1]!=-1 and x[2]!=-1)]
bounds.sort()
record_list("Dark_Photon_Seaquest_low_POT.dat",bounds)

In [22]:
files1=glob.glob('Visible_Dark_Photon/seaquest/seaquest_*')

In [27]:
calc_params('Visible_Dark_Photon/seaquest/seaquest_10_1e-07.dat',Seaquest_evnt,Seaquest_Cut_no_kmag,Seaquest_Cut_2_no_kmag)

-93.1956635090176


In [32]:
calc_params('Visible_Dark_Photon/seaquest/seaquest_10_1e-07.dat',Seaquest_evnt,Seaquest_Cut,Seaquest_Cut_2)

-23.406551488467052


In [28]:
calc_params('Visible_Dark_Photon/seaquest_extended/seaquest_10_1e-07.dat',Seaquest_evnt,Seaquest_Cut_no_kmag,Seaquest_Cut_2_no_kmag)

-131.6800557431009


In [31]:
5.69/5.24

1.08587786259542

In [26]:
files=glob.glob('Visible_Dark_Photon/seaquest_extended/seaquest_*')
bounds=[calc_params(file,Seaquest_evnt,Seaquest_Cut,Seaquest_Cut_2) for file in files]
bounds = [x for x in bounds if x[0]!=-1 and (x[1]!=-1 and x[2]!=-1)]
bounds.sort()
record_list("Dark_Photon_Seaquest_Extended_with_KMAG.dat",bounds)

In [27]:
calc_params(files[0],NA62_evnt,NA62_Cut)

[0.11, 1.4940476726345034e-07, 1.4484081912125851e-05]

In [38]:
dat2=Event_Parser(files[1], NA62_Cut)

In [57]:
files=glob.glob("Visible_Dark_Photon/NA62/*")

In [42]:
files=glob.glob("Visible_Dark_Photon/NA62/*")
bounds=[calc_params(file,NA62_evnt,NA62_Cut) for file in files]
bounds = [x for x in bounds if x[0]!=-1 and (x[1]!=-1 and x[2]!=-1)]
bounds.sort()
record_list("Dark_Photon_NA62.dat",bounds)

In [21]:
files=glob.glob("Visible_Dark_Photon/CHARM/charm_decays_*")
bounds=[calc_params(file,charm_evnt,CHARM_Cut) for file in files]
bounds = [x for x in bounds if x[0]!=-1 and (x[1]!=-1 and x[2]!=-1)]
bounds.sort()
record_list("Dark_Photon_CHARM.dat",bounds)

In [16]:
files=glob.glob("Visible_Dark_Photon/nucal/*")
bounds=[calc_params(file,Nucal_evnt_1,Nucal_Cut_1) for file in files]
bounds = [x for x in bounds if x[0]!=-1 and (x[1]!=-1 and x[2]!=-1)]
bounds.sort()
record_list("Dark_Photon_nucal_3gev.dat",bounds)

In [17]:
files=glob.glob("Visible_Dark_Photon/nucal/*")
bounds=[calc_params(file,Nucal_evnt_2,Nucal_Cut_2) for file in files]
bounds = [x for x in bounds if x[0]!=-1 and (x[1]!=-1 and x[2]!=-1)]
bounds.sort()
record_list("Dark_Photon_nucal_10gev.dat",bounds)